In [1]:
import itertools
import numpy as np
from typing import Tuple, List
import pandas as pd

In [2]:
def mult_well(values: np.ndarray):
    if 0 in values:
        return 0
    sign = 1 if len([v for v in values if v < 0]) % 2 == 0 else -1
    values = np.abs(values)
    less_than_one = sorted(values[values < 1])
    greater_than_one = sorted(values[values > 1], reverse=True)
    prod = 1
    while len(less_than_one) + len(greater_than_one):
        if len(less_than_one) == 0:
            prod *= np.prod(greater_than_one)
            break
        elif len(greater_than_one) == 0:
            prod *= np.prod(less_than_one)
            break
        elif prod > 1:
            small_value = less_than_one.pop(0)
            prod *= small_value
        else:  # prod < 1
            large_value = greater_than_one.pop(0)
            prod *= large_value
    return sign * prod

In [3]:
def get_g_lazy(S, n, k) -> np.ndarray:
    return np.array([1 / S] * (k - 1) + [1 - 1 / S] * (n - k))


def get_comb_lazy(n, k) -> np.ndarray:
    values = []
    values += list(range(n, n - k, -1))
    values += list(1 / np.arange(1, k + 1))
    return np.array(values)


def get_f_lazy(S, n, k) -> np.ndarray:
    return np.concatenate([get_g_lazy(S, n, k), get_comb_lazy(n, k)])

In [4]:
def get_var_lazy(S, n, k) -> Tuple[Tuple[int, ...], List[np.ndarray]]:
    fk_lazy = get_f_lazy(S, n, k)
    fksquare_lazy = fk_lazy**2
    if 2 * k <= n:
        values = []
        values += list(range(n, n - 2 * k, -1))
        values += list(1 / np.arange(1, k + 1))
        values += list(1 / np.arange(1, k + 1))
        values += [S - 2] * (n - 2 * k)
        values += [S - 1]
        values += [1 / S] * (n - 1)
        values = np.array(values)
        return (1, -1, 1), [fk_lazy, fksquare_lazy, values]
    else:
        return (1, -1), [fk_lazy, fksquare_lazy]


def get_cov_lazy(S, n, k1, k2) -> Tuple[Tuple[int, ...], List[np.ndarray]]:
    fk1_lazy = get_f_lazy(S, n, k1)
    fk2_lazy = get_f_lazy(S, n, k2)
    fk1k2_lazy = np.concatenate([fk1_lazy, fk2_lazy])
    # cov = - mult_well(values)
    if k1 + k2 <= n:
        values = []
        values += list(range(n, n - k1 - k2, -1))
        values += list(1 / np.arange(1, k1 + 1))
        values += list(1 / np.arange(1, k2 + 1))
        values += [S - 2] * (n - k1 - k2)
        values += [S - 1]
        values += [1 / S] * (n - 1)
        values = np.array(values)
        return (-1, 1), [fk1k2_lazy, values]
    else:
        return (-1,), [fk1k2_lazy]

In [5]:
def get_var_bt_lazy(S, n) -> float:
    var_bt = 0
    for i in range(1, n + 1):
        ci_values = 1 / get_comb_lazy(n, i)
        cisquare_values = ci_values**2
        var_tuples = get_var_lazy(S, n, i)
        for coef, var_values in zip(var_tuples[0], var_tuples[1]):
            values = np.concatenate([cisquare_values, var_values])
            var_bt += coef * mult_well(values)
    for i, j in itertools.combinations(range(1, n + 1), 2):
        sign = 1 if i + j % 2 == 0 else -1
        ci_values = 1 / get_comb_lazy(n, i)
        cj_values = 1 / get_comb_lazy(n, j)
        cov_tuples = get_cov_lazy(S, n, i, j)
        for coef, cov_values in zip(cov_tuples[0], cov_tuples[1]):
            values = np.concatenate([ci_values, cj_values, cov_values])
            var_bt += sign * coef * mult_well(values)
    return var_bt

In [6]:
def get_var_gt(S, n):
    var1_tuple = get_var_lazy(S, n, 1)
    var1 = 0
    for coef, values in zip(var1_tuple[0], var1_tuple[1]):
        var1 += coef * mult_well(values)
    return var1 / (n**2)

In [7]:
def get_bias_gt(S, n):
    return mult_well(get_g_lazy(S, n + 1, 2))

def get_bias_bt(S, n):
    return mult_well(get_g_lazy(S, n + 1, n + 1))

In [8]:
# roughly 45 minutes
Sns = [(100, 100), (100, 500), (100, 1000)]

data = []
for S, n in Sns:
    print(f"S = {S}, n = {n}")
    data.append([S, n, "GT", get_bias_gt(S, n), get_var_gt(S, n)])
    print(f"GT: bias = {data[-1][-2]}, var = {data[-1][-1]}")
    data.append([S, n, "BT", get_bias_bt(S, n), get_var_bt_lazy(S, n)])
    print(f"BT: bias = {data[-1][-2]}, var = {data[-1][-1]}")

S = 100, n = 100
GT: bias = 0.003697296376497265, var = 0.00233717764693788
BT: bias = 1.0000000000000017e-200, var = 0.0023515371697249926
S = 100, n = 500
GT: bias = 6.636851557994564e-05, var = 1.1429692313457735e-05
BT: bias = 0.0, var = 1.1445362183927047e-05
S = 100, n = 1000
GT: bias = 4.360732061682617e-07, var = 4.343882460648888e-08
BT: bias = 0.0, var = 4.344124759226334e-08


In [9]:
df = pd.DataFrame(data, columns=["S", "n", "method", "bias", "var"])
df["mse"] = df["bias"] ** 2 + df["var"]
display(df)

,S,n,method,bias,var,mse
0,100,100,GT,3.697296e-03,2.337178e-03,2.350848e-03
1,100,100,BT,1.000000e-200,2.351537e-03,2.351537e-03
2,100,500,GT,6.636852e-05,1.142969e-05,1.143410e-05
3,100,500,BT,0.000000e+00,1.144536e-05,1.144536e-05
4,100,1000,GT,4.360732e-07,4.343882e-08,4.343901e-08
5,100,1000,BT,0.000000e+00,4.344125e-08,4.344125e-08


In [10]:
# as latex
print(df.to_latex(index=False,float_format="{:.4e}".format))

\begin{tabular}{rrlrrr}
\toprule
S & n & method & bias & var & mse \\
\midrule
100 & 100 & GT & 3.6973e-03 & 2.3372e-03 & 2.3508e-03 \\
100 & 100 & BT & 1.0000e-200 & 2.3515e-03 & 2.3515e-03 \\
100 & 500 & GT & 6.6369e-05 & 1.1430e-05 & 1.1434e-05 \\
100 & 500 & BT & 0.0000e+00 & 1.1445e-05 & 1.1445e-05 \\
100 & 1000 & GT & 4.3607e-07 & 4.3439e-08 & 4.3439e-08 \\
100 & 1000 & BT & 0.0000e+00 & 4.3441e-08 & 4.3441e-08 \\
\bottomrule
\end{tabular}

